In [1]:
using Oceananigans
using CairoMakie
using JLD2
using Statistics

[ Info: Oceananigans will use 4 threads


In [9]:
foldername = "../../scratch/filament-instability/Ri000-3d"
filename = "3d_state.jld2"
paramfilename = "parameters.jld2"
sp = jldopen("$foldername/$paramfilename") do file
    file["parameters/simulation"]
end
frames, grid = jldopen("$foldername/$filename") do file
    (keys(file["timeseries/t"])[1:end], file["serialized/grid"])
end
ts = jldopen("$foldername/$filename") do file
    [file["timeseries/t/$frame"] for frame in frames] .- 1
end
xᶜᵃᵃ = xnodes(Center, grid)
xᶠᵃᵃ = xnodes(Face, grid)
zᵃᵃᶜ = znodes(Center, grid)
zᵃᵃᶠ = znodes(Face, grid)
Δzᵃᵃᶜ = reshape(diff(zᵃᵃᶠ), 1, 1, length(zᵃᵃᶜ))
Δx = xᶠᵃᵃ[2] - xᶠᵃᵃ[1]

@inline function ∂x(fᶜᵃᵃ)
    return (circshift(fᶜᵃᵃ, (-1, 0, 0)) - circshift(fᶜᵃᵃ, (1, 0, 0))) / (2Δx)
end
@inline function ∂z(fᶜᵃᵃ)
    let a = (circshift(fᶜᵃᵃ, (0, 0, -1)) - circshift(fᶜᵃᵃ, (0, 0, 1))) ./ (2Δzᵃᵃᶜ)
        a[:, :, 1] .= 0
        a[:, :, end] .= 0
        a
    end
end

∂z (generic function with 1 method)

In [10]:
t = 2.32
i = argmin(abs.(ts .- t))
frame = frames[i]

"1704"

In [11]:
ω = jldopen("$foldername/$filename") do file
    u = file["timeseries/u/$frame"]
    u = (circshift(u, (-1, 0, 0)) .+ circshift(u, (1, 0, 0))) / 2
    w = file["timeseries/w/$frame"]
    w = (w[:, :, 1:end-1] .+ w[:, :, 2:end]) / 2
    ∂z(u) - ∂x(w)
    end;

In [12]:
size(ω)

(1024, 1024, 128)

In [14]:
v′w′ = jldopen("$foldername/$filename") do file
    v = file["timeseries/v/$frame"]
    v = (circshift(v, (0, -1, 0)) .+ circshift(v, (0, 1, 0))) / 2
    w = file["timeseries/w/$frame"]
    w = (w[:, :, 1:end-1] .+ w[:, :, 2:end]) / 2
    
    v′ = v .- mean(w; dims=2)
    w′ = w .- mean(w; dims=2)
    
    v′ .* w′
    end;

In [15]:
jldopen("../3D$frame.jld2", "a+") do file
    file["ω"] = ω
    file["v′w′"] = v′w′
    end;